### Proving Theorems In Propositional Logic with LSTM-Based Text Generators
#### Author: Omar Afifi

Consider a number of propositional (i.e. variable-free) sentences (premsises). For example: 

1. p→q 
2. o
3. pv¬o

A propositional proof from the premises to a conclusion (another sentence) is a sequence of variable-free statements that follow from the premises by logical deduction rules (e.g. modus ponens, modus tollens, modus tollendo ponens, dysjuntive elimination, etc ... )

For example, a proof of the propositional sentence (q) from the preceding premises is as follows: 

4. ¬¬o (from 2, double negation)
5. p (from 3 and 4, dysjuntive elimination )
5. q (from 1 and 5, modus ponens)

QED.


This notebook explores the utility of using LSTM text-generators to generate a propositional proof given a collection of propositional sentences. Our hope is that it can be helpful as a stepping stone to making progress in the arena of stochastic theorem provers. 

Credits: Hugging Face User ergotts for building this dataset: https://huggingface.co/datasets/ergotts/propositional-logic


### Loading Data and preparring the input

In [1]:
import process_data

#load the data from hugging face mode = 'w' means that we are tokenizing words rather than characters or sentences. 
proofs_dataset = process_data.LoadLogicData(mode = 'w') 

#format the proofs: essentially just mapping words to integers and then creating n-gram sequences
word_to_int, int_to_word, sequenced_proofs = process_data.generate_sequences(proofs_dataset)
word_to_int[' '] = 0; int_to_word[0] = ' '

# pad the sequences 
sequenced_proofs = process_data.pad(sequenced_proofs, value = 0)
#split data into input and label by setting label equal to next word.
X,y = process_data.makeXy(sequenced_proofs)


/Users/omarafifi/Downloads/LSTM-Prover/Automated-Theorem-Proving-With-LSTM-Networks/lstm_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### making the data compatible with torch API

In [2]:
import torch as t
import torch.utils.data  as data

X = t.tensor(X, dtype = t.int64)
y = t.tensor(y, dtype = t.int64)

torch_data = data.DataLoader(data.TensorDataset(X,y),
                            batch_size = 100)

### Loading the Model and Training

In [ ]:
import torch
import model

vocab_size = len(word_to_int)
hidden_size = 2
num_layers = 2
loss_function = torch.nn.CrossEntropyLoss(reduction = "mean")



lstm = model.LSTM(seq_length = len(X[0]), 
            hidden_size = hidden_size, 
            num_layers = num_layers, 
            vocab_size = vocab_size)


optimizer = torch.optim.Adam(params = lstm.parameters(), lr = .0001)
lstm.train(torch_data, 1000, loss_function, optimizer)
